In [1]:
from selenium import webdriver
import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 함수로 만들기
def get_articles(page):
    driver =  webdriver.Chrome()
    url = 'https://search.zdnet.co.kr/?kwd=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&page={}'.format(page)
    driver.get(url)
    #여러데이터 모으기
    articles = driver.find_elements_by_css_selector("body > div.contentWrapper > div > div.left_cont > section > div")
    
    #제목
    title_list = []
    for article in articles:
        title = article.find_element_by_css_selector("div.assetText > a > h3").text 
        title_list.append(title)
    title =pd.DataFrame(title_list, columns=['title'])
    
    #링크만 뽑아내기
    link_list = []
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        link_list.append(link)
    link= pd.DataFrame(link_list, columns=['link'])
    
    #상세내용 뽑아내기
    preview_list = []
    for article in articles:
        preview = article.find_element_by_css_selector("div.assetText > a > p").text[:200]
        preview_list.append(preview)
    preview=pd.DataFrame(preview_list,columns=['preview'])
    
    result=pd.concat([title, link,preview], axis=1)
    driver.quit()
    return result

# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def zd_net(startpage, endpage):
    import pandas as pd
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_articles(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df
final_df= zd_net(1,2)      

In [3]:
final_df.to_csv("csv_files/zd_net_bigdata.csv", index=False, encoding='utf-8' )


In [18]:
url= 'http://www.itworld.co.kr/t/54649/%EB%B9%85%20%EB%8D%B0%EC%9D%B4%ED%84%B0?page=2'
domain = 'http://www.itworld.co.kr'
# headers = {
#     "user-agent": UserAgent().chrome,
#     "referer": "https://finance.daum.net/exchanges",
# }
response= requests.get(url)
response

<Response [200]>

In [19]:
# 2. BeautifulSoup을 이용하여 css selector를 사용할 수 있는 객체로 파싱
# dom은 HTML 전체를 뷰티풀숩이 객체로 가져온것
dom = BeautifulSoup(response.content, "html.parser")
# 3. css selector를 이용하여 원하는 데이터 수집
# select : 여러개의 엘리먼트를 리스트로 가져옴
# select_one : 하나의 엘리먼트 객체를 가져옴
# dom

In [22]:
articles =dom.select('.news_list_')
articles[0].select_one('.news_list_has_thumb_size > .news_body_summary').text.replace("\n", "").replace("\t", "").strip()

'대규모 병렬 처리(Massively Parallel Processing, MPP) 데이터베이스는 데이터와 쿼리를 일반 서버로 구성된 클러스터의 각 노드로 분산시킨다. 그린플럼(Greenplum)의 MPP 데이터 웨어하우스 구축 방법은 독특하다. 기존  ...'

In [13]:
datas = []

for article in articles:
    try:
        preview = article.select_one('.news_list_has_thumb_size > .news_body_summary').text.replace("\n", "").replace("\t", "").strip()
    except:
        preview = article.select_one('.news_list_full_size > .news_body_summary').text.replace("\n", "").replace("\t", "").strip()
    datas.append({
        "title" : article.select_one('div > h4 > a').text,
        "link" : domain + article.select_one('div > h4 > a')['href'],
        "preview": preview
    })

bigdata_df = pd.DataFrame(datas)
bigdata_df.tail()

,title,link,preview
10,"몽고DB, 알리바바 클라우드와 파트너십 체결",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,몽고DB가 ‘서비스로서의 몽고DB(MongoDB-as-a-Service)’ 솔루션을...
11,"“미래는 나의 것” NoSQL, 뉴 스택 소프트웨어의 확실한 선두주자",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,"관계형 데이터베이스는 지난 몇십 년 동안 엄청난 인기를 누렸지만, 모건 스탠리의 최..."
12,"""경기날 음료 판매량까지 정확히 예측""··· NBA 마이애미히트의 애널리틱스 사례",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,오늘날 스포츠 팬은 스마트폰 덕분에 행복하다. 다양한 모바일 앱에 통해 더 편리하게...
13,"롯데정보통신, SAP 퀄트릭스 파트너 네트워크 합류...“고객경험 분야 서비스 역량...",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,롯데정보통신은 ‘퀄트릭스 파트너 네트워크(Qualtrics Partner Netwo...
14,'데이터 속 가치 찾기' 빅데이터 분석의 정의와 활용도,http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,데이터는 무엇이고 빅데이터는 과연 무엇일까? 그리고 그 두 가지의 차이를 확실히 구...


In [4]:
#Techcrunch
def get_itworld(page):
    
    url= 'http://www.itworld.co.kr/t/54649/%EB%B9%85%20%EB%8D%B0%EC%9D%B4%ED%84%B0?page={}'.format(page)
    domain = 'http://www.itworld.co.kr'

    response= requests.get(url)
    # 2. BeautifulSoup을 이용하여 css selector를 사용할 수 있는 객체로 파싱
    # dom은 HTML 전체를 뷰티풀숩이 객체로 가져온것
    dom = BeautifulSoup(response.content, "html.parser")
    articles =dom.select('.news_list_')
    
    datas = []

    for article in articles:
        try:
            preview = article.select_one('.news_list_has_thumb_size > .news_body_summary').text.replace("\n", "").replace("\t", "").strip()
        except:
            preview = article.select_one('.news_list_full_size > .news_body_summary').text.replace("\n", "").replace("\t", "").strip()
        datas.append({
            "title" : article.select_one('div > h4 > a').text,
            "link" : domain + article.select_one('div > h4 > a')['href'],
            "preview": preview
        })

    bigdata_df = pd.DataFrame(datas)
    return bigdata_df

# 자동으로 일정 페이지 크롤링해줘서 데이터프레임 만들어주기
def Tech_itworld(startpage, endpage):
    import pandas as pd
    df=pd.DataFrame([])
    for i in range(startpage, endpage+1):
        temp=get_itworld(i)
        df= pd.concat([df,temp],axis=0)
    #index 번호 다시 매기기
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
final_df2= Tech_itworld(1,2)
final_df2.to_csv("csv_files/Tech_itworld_bigdata.csv", index=False, encoding='utf-8' )

In [6]:
import pymongo
import pandas as pd
import requests

# 데이터베이스 서버 연결
client = pymongo.MongoClient("mongodb://localhost:27017/")

#컬렉션 삭제
client.crawling.drop_collection("news_db_bigdata")

# 데이터 베이스와 컬렉션 생성
#서버.데이터베이스.컬렉션
news_db_bigdata = client.crawling.news_db_bigdata


final_df=pd.read_csv("csv_files/zd_net_bigdata.csv",encoding="utf-8")
final_df2=pd.read_csv("csv_files/Tech_itworld_bigdata.csv",encoding="utf-8")

items1= final_df.to_dict("records")
items2= final_df2.to_dict("records")

ids=news_db_bigdata.insert(items1)
ids=news_db_bigdata.insert(items2)